# Verying base line for paper Identifying causal gateways and mediators in complex spatio-temporal system

## Abstract

In a complex spatio-temporal system such as Earth's climate, extream natural event or geo engineering can have a spreading or mediating perturbations through causal gateway regions. In this article we will check accuracy and establish baseline done in this work [ 1 ](https://www.nature.com/articles/ncomms9502). We will first explore the process of data gathering, then analyize the data, reduce the dimension for processing and finally visit causal reconstruction.


## Data collection

Data is collected through [NOAA Physical Sciences labaratory](https://psl.noaa.gov/data/gridded/data.ncep.reanalysis.surface.html) under NCEP/NCAR Reanalysis. Since the amount of data we are dealing with is vast I have written a small script download_data.py for downloading all the data we will need through concurrent threads. It is recommened to use an external hard drive fo data collection.

In [1]:
print('download_data')

download_data


## Data analysis

The data we collected in the previous step are in cdf format and we will use python netCDF4 package to handle the data. In climate research, spatio-temporal data sets are typically given on a regular grid. Here we consider a reanalysis data set of surface pressures for period 9448-2012. At a resolution of $2.5^o$ in latitude and longitude. As an introductory data analysis we will also consider surface air temperature data.

In [11]:
# documentation: https://unidata.github.io/netcdf4-python/
import pandas as pd
import geopandas
import datetime
from netCDF4 import Dataset
d = Dataset('/Users/naveenmysore/Downloads/air.1948.nc')

In [12]:
print("--------")
print(d.title)
print("--------")
print(d.description)
print("--------")
print(d.variables)

--------
4x daily NMC reanalysis (1948)
--------
Data is from NMC initialized reanalysis
(4x/day).  It consists of most variables interpolated to
pressure surfaces from model (sigma) surfaces.
--------
{'level': <class 'netCDF4._netCDF4.Variable'>
float32 level(level)
    units: millibar
    actual_range: [1000.   10.]
    long_name: Level
    positive: down
    GRIB_id: 100
    GRIB_name: hPa
    axis: Z
unlimited dimensions: 
current shape = (17,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'lat': <class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
    units: degrees_north
    actual_range: [ 90. -90.]
    long_name: Latitude
    standard_name: latitude
    axis: Y
unlimited dimensions: 
current shape = (73,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'lon': <class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
    units: degrees_east
    long_name: Longitude
    actual_range: [  0.  357.5]
    standard_name: longitude
    axis: X
unlimited dimens

In [34]:
longitudes = d.variables['lon'][:]
latitudes = d.variables['lat'][:]
levels = d.variables['level'][:]
time_stamps = d.variables['time'][:]
temperatures = d.variables['air'][:]
dates = []
date_time = datetime.datetime(1948, 1, 1, 0, 0, 0)
_data = []
for i in range(0, len(time_stamps)):
    _data.append(temperatures[i][0])
    dates.append(date_time)
    date_time += datetime.timedelta(hours=6)
print(len(temperatures))
print(len(_data))
print(len(longitudes))
print(len(latitudes))


1464
1464
144
73


In [33]:
df = pd.DataFrame(
    {'City': ['Buenos Aires', 'Brasilia', 'Santiago', 'Bogota', 'Caracas'],
     'Country': ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Venezuela'],
     'Latitude': [-34.58, -15.78, -33.45, 4.60, 10.48],
     'Longitude': [-58.66, -47.91, -70.66, -74.08, -66.86]})
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))
print(gdf)

df = pd.DataFrame(
    {'temperatures': d.variables['air'][:],
     'Latitude': d.variables['lat'][:],
     'Longitude': d.variables['lon'][:]})
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))
print(gdf)


           City    Country  Latitude  Longitude                     geometry
0  Buenos Aires  Argentina    -34.58     -58.66  POINT (-58.66000 -34.58000)
1      Brasilia     Brazil    -15.78     -47.91  POINT (-47.91000 -15.78000)
2      Santiago      Chile    -33.45     -70.66  POINT (-70.66000 -33.45000)
3        Bogota   Colombia      4.60     -74.08    POINT (-74.08000 4.60000)
4       Caracas  Venezuela     10.48     -66.86   POINT (-66.86000 10.48000)


ValueError: arrays must all be same length

In [30]:
data = {}
data['latitudes'] = latitudes
data['longitudes'] = longitudes
data['levels'] = levels
data['time_stamps_ordinal'] = time_stamps
data['time_stamps'] = dates
data['air'] = temperatures

_df = pd.Series(data, index=dates, name='air_temperature')
print(_df) 


1948-01-01 00:00:00    NaN
1948-01-01 06:00:00    NaN
1948-01-01 12:00:00    NaN
1948-01-01 18:00:00    NaN
1948-01-02 00:00:00    NaN
                      ... 
1948-12-30 18:00:00    NaN
1948-12-31 00:00:00    NaN
1948-12-31 06:00:00    NaN
1948-12-31 12:00:00    NaN
1948-12-31 18:00:00    NaN
Name: air_temperature, Length: 1464, dtype: object


In [24]:
print(d.variables['level'])

<class 'netCDF4._netCDF4.Variable'>
float32 level(level)
    units: millibar
    actual_range: [1000.   10.]
    long_name: Level
    positive: down
    GRIB_id: 100
    GRIB_name: hPa
    axis: Z
unlimited dimensions: 
current shape = (17,)
filling on, default _FillValue of 9.969209968386869e+36 used


In [25]:
gdf = geopandas.GeoDataFrame(_df, geometry=geopandas.points_from_xy(_df.longitudes, _df.latitudes))

AttributeError: 'Series' object has no attribute 'longitudes'

In [18]:
print(d.variables['air'])

<class 'netCDF4._netCDF4.Variable'>
float32 air(time, level, lat, lon)
    long_name: 4xDaily Air temperature
    units: degK
    precision: 2
    least_significant_digit: 1
    GRIB_id: 11
    GRIB_name: TMP
    var_desc: Air temperature
    level_desc: Multiple levels
    statistic: Individual Obs
    parent_stat: Other
    missing_value: -9.96921e+36
    actual_range: [150.  326.3]
    valid_range: [150. 350.]
    dataset: NCEP Reanalysis
unlimited dimensions: time
current shape = (1464, 17, 73, 144)
filling on, default _FillValue of 9.969209968386869e+36 used


In [19]:
count = 1464*17*73*144
print(count)

261622656


In [31]:
data = {}
print(len(latitudes))
print(len(longitudes))
date_time = datetime.datetime(1948, 1, 1, 0, 0, 0)

print(d.variables['air'][0][0].shape)

start = time.time()
for k in range(0, len(times)):
    for i in range(0, len(latitudes)):
        for j in range(0, len(longitudes)):
            data['time'] = date_time
            data['time_stamp'] = times[k]
            data['latitude'] = latitudes[i]
            data['longitude'] = longitudes[j]
            data['temperature'] = d.variables['air'][k][0][i][j]
            date_time += datetime.timedelta(hours=6)
end = time.time()
print(f'{end - start} elapsed')
print(data)

73
144
(73, 144)


KeyboardInterrupt: 